In [ ]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ruta absoluta del archivo NetCDF en el clúster
file_path = "datos/Land_and_Ocean_EqualArea.nc"

data = xr.open_dataset(file_path)
latitudes = data['latitude'].values
longitudes = data['longitude'].values
temperature = data['temperature'].values

times = data['time'].values
lat_grid = np.linspace(latitudes.min(), latitudes.max(), 100)  # Ajusta la resolución de la cuadrícula según sea necesario
lon_grid = np.linspace(longitudes.min(), longitudes.max(), 100)
lon_grid_mesh, lat_grid_mesh = np.meshgrid(lon_grid, lat_grid)

# Definir las décadas basadas en la variable de tiempo (asumiendo que el tiempo está en años)
decades = np.floor(times / 10) * 10

# Inicializar un diccionario para almacenar las matrices de correlación por década
correlation_matrices_by_decade = {}

# Procesar datos por década
unique_decades = np.unique(decades)

# Crear un directorio para guardar los datos
save_dir = "decade_correlations_lag_3"
os.makedirs(save_dir, exist_ok=True)

for decade in unique_decades:
    # Extraer datos para la década actual
    decade_mask = (decades == decade)
    temperature_decade = temperature[decade_mask, :]

    # Interpolar los datos en una cuadrícula regular para todos los puntos de tiempo en la década
    temperature_grid_over_time_decade = griddata(
        (latitudes.flatten(), longitudes.flatten()),
        temperature_decade.reshape(-1),
        (lon_grid_mesh, lat_grid_mesh),
        method='linear'
    ).reshape(len(temperature_decade), -1)  # Forma (N, 10000)

    # Verificar si hay suficientes puntos para el lag
    if temperature_grid_over_time_decade.shape[0] <= 3:
        correlation_matrices_by_decade[int(decade)] = np.full((temperature_grid_over_time_decade.shape[1], temperature_grid_over_time_decade.shape[1]), np.nan)
        np.save(os.path.join(save_dir, f'correlation_matrix_lag3_{int(decade)}.npy'), correlation_matrices_by_decade[int(decade)])
        continue

    # Crear matrices desplazadas para el lag
    X = temperature_grid_over_time_decade[:-3, :]  # Forma (N-3, M)
    Y = temperature_grid_over_time_decade[3:, :]   # Forma (N-3, M)

    # Calcular la matriz de correlación utilizando Numpy de manera vectorizada
    # Normalizar las matrices
    X_mean = X.mean(axis=0)
    Y_mean = Y.mean(axis=0)
    X_std = X.std(axis=0)
    Y_std = Y.std(axis=0)

    # Evitar división por cero
    X_std[X_std == 0] = 1
    Y_std[Y_std == 0] = 1

    X_normalized = (X - X_mean) / X_std
    Y_normalized = (Y - Y_mean) / Y_std

    # Calcular la matriz de correlación
    correlation_matrix_decade_lag3 = np.dot(X_normalized.T, Y_normalized) / (X.shape[0] - 1)

    # Almacenar la matriz de correlación en el diccionario
    correlation_matrices_by_decade[int(decade)] = correlation_matrix_decade_lag3

    # Guardar la matriz de correlación para esta década
    np.save(os.path.join(save_dir, f'correlation_matrix_lag3_{int(decade)}.npy'), correlation_matrix_decade_lag3)
    
# Guardar la lista de décadas
saved_decades = list(correlation_matrices_by_decade.keys())
np.save(os.path.join(save_dir, 'saved_decades.npy'), np.array(saved_decades))